In [8]:
from Bio import SeqIO
import numpy as np
from utils import *


In [4]:
transcript_file = np.genfromtxt('./data/GENCODE_v34_hg38_comprehensive', usecols=(1, 2, 3, 4, 5, 9, 10), skip_header=1, dtype='str')
DE_tr = np.genfromtxt('./lists/limma_DE_AML_RAN', usecols=(1,), skip_header=1, dtype='str')

print(DE_tr)

['ENST00000257818.2' 'ENST00000241453.11' 'ENST00000216336.2'
 'ENST00000284509.10' 'ENST00000592205.5' 'ENST00000633060.1'
 'ENST00000427103.5' 'ENST00000360121.4' 'ENST00000378962.3'
 'ENST00000612677.4' 'ENST00000309017.7' 'ENST00000304625.2'
 'ENST00000380987.2' 'ENST00000598473.1' 'ENST00000233997.3'
 'ENST00000620695.2' 'ENST00000367279.8' 'ENST00000376581.9'
 'ENST00000635923.1' 'ENST00000448387.6' 'ENST00000537784.5'
 'ENST00000563039.2' 'ENST00000381297.9' 'ENST00000611771.1'
 'ENST00000430686.2' 'ENST00000304639.3' 'ENST00000393118.6'
 'ENST00000554578.5' 'ENST00000400007.8' 'ENST00000245479.2'
 'ENST00000561385.5' 'ENST00000215855.6' 'ENST00000293373.10'
 'ENST00000468385.1' 'ENST00000477988.1' 'ENST00000282026.1'
 'ENST00000346128.10' 'ENST00000261233.8' 'ENST00000359135.7'
 'ENST00000367814.8' 'ENST00000515859.5' 'ENST00000507316.1'
 'ENST00000355530.6' 'ENST00000531348.5' 'ENST00000262262.4'
 'ENST00000264824.4' 'ENST00000527615.5' 'ENST00000381501.7'
 'ENST00000373304.3'

In [6]:
fasta_seq = SeqIO.parse(open('./data/hg38.fa'), 'fasta')

for fasta in fasta_seq:
    name, sequence = fasta.id, str(fasta.seq)

In [24]:
transcripts = []
labels = []

# flanking ends on each side are of this length to include some context
context = 1000

for row in transcript_file:
    # explicitly checking transcript_name
    if row[0] in DE_tr:
        # sequence from start to end
        s = sequence[int(row[3]) - context: int(row[4]) + context].upper()
        # adding the transcripts of the sense strand: whole transcript + flanks + zero-padded, labels + zero-padded
        if row[2] == '+':
            # extract the transcript sequence with 1k flanks
            if 'N' not in s:
                # padding labels here
                pad = 5000 - (len(s) - context * 2) % 5000
                es, ee = row[5].split(',')[:-1], row[6].split(',')[:-1]
                # decrease the pad length from both sides because the context-1 and context+sequence+1 sites are
                # donor and acceptor, respectively
                y = make_labels(s, context, es, ee)
                labels.append(y)
                # padding sequence with Os
                s = (pad // 2) * 'O' + s + (pad - pad // 2) * 'O'
                transcripts.append(s)
            else:
                print('contains N')
        # adding the transcripts of the antisense strand
        if row[2] == '-':
            if 'N' not in s:
                # padding labels here
                pad = 5000 - (len(s) - context * 2) % 5000
                # decrease the pad length from both sides because the context-1 and context+sequence+1 sites are
                # donor and acceptor, respectively
                es, ee = row[5].split(',')[:-1], row[6].split(',')[:-1]
                # decrease the pad length from both sides because the context-1 and context+sequence+1 sites are
                # donor and acceptor, respectively
                y = make_labels(s, context, es, ee)
                labels.append(y)
                # hot-encoding labels and adding hot-encoded labels to a new list
                # getting complementary seq
                s = ''.join([complementary(x) for x in s])
                # padding sequence with Os
                s = (pad // 2) * 'O' + s + (pad - pad // 2) * 'O'
                transcripts.append(s)
            else:
                print('contains N')


In [47]:
def chr_to_N(s):
    if s[3:]=='X':
        N = 22
    elif s[3:]=='Y':
        N = 23
    else:
        N = int(s[3:]) - 1
        
    return N
        
a = ['chr1', 'chr3', 'chr22', 'chr12', 'chrY']
print([chr_to_N(x) for x in a])

chr_len = np.genfromtxt('./data/chr_len_hg38.txt', usecols=(0,), dtype='int')
print(chr_len)
print(np.sum(chr_len[:10]))

[0, 2, 21, 11, 23]
[248956422 242193529 198295559 190214555 181538259 170805979 159345973
 145138636 138394717 133797422 135086622 133275309 114364328 107043718
 101991189  90338345  83257441  80373285  58617616  64444167  46709983
  50818468 156040895  57227415]
1808681051


In [51]:
print(len(sequence))

37240


In [45]:
for t in DE_tr[0:1]:
    found = 0
    # explicitly checking transcript_name
    for row in transcript_file:
        if row[0][:15]==t[:15]:
            found = 1
            print(row[0])
            print(row[3], row[4])
            print(row[1])
            chrN = chr_to_N(row[1])
            start = np.sum(chr_len[:chrN])
            s = sequence[int(row[3]) + start - context: int(row[4]) + start + context].upper()
            print(int(row[3]) + start - context, int(row[3]) + start + context)
            print(s)
            # adding the transcripts of the sense strand: whole transcript + flanks + zero-padded, labels + zero-padded
            if row[2] == '+':
                # extract the transcript sequence with 1k flanks
                if 'N' not in s:
                    # padding labels here
                    pad = 5000 - (len(s) - context * 2) % 5000
                    es, ee = row[5].split(',')[:-1], row[6].split(',')[:-1]
                    # decrease the pad length from both sides because the context-1 and context+sequence+1 sites are
                    # donor and acceptor, respectively
                    y = make_labels(s, context, es, ee)
                    labels.append(y)
                    # padding sequence with Os
                    s = (pad // 2) * 'O' + s + (pad - pad // 2) * 'O'
                    transcripts.append(s)
                else:
                    print('contains N')
            # adding the transcripts of the antisense strand
            if row[2] == '-':
                if 'N' not in s:
                    # padding labels here
                    pad = 5000 - (len(s) - context * 2) % 5000
                    # decrease the pad length from both sides because the context-1 and context+sequence+1 sites are
                    # donor and acceptor, respectively
                    es, ee = row[5].split(',')[:-1], row[6].split(',')[:-1]
                    # decrease the pad length from both sides because the context-1 and context+sequence+1 sites are
                    # donor and acceptor, respectively
                    y = make_labels(s, context, es, ee)
                    labels.append(y)
                    # hot-encoding labels and adding hot-encoded labels to a new list
                    # getting complementary seq
                    s = ''.join([complementary(x) for x in s])
                    # padding sequence with Os
                    s = (pad // 2) * 'O' + s + (pad - pad // 2) * 'O'
                    transcripts.append(s)
                else:
                    print('contains N')
    if found==0:
        print(t, 'not found')


ENST00000257818.3
33858575 33892076
chr11
1842538626 1842540626



In [33]:
print(len(transcripts))
print(len(DE_tr))

print(transcript_file[0][0][:15])
if 'ENST00000281938.6' in DE_tr:
    print('yeth')
    
print(transcripts[0]==transcripts[1])
print(transcripts[1]==transcripts[2])
print(transcripts[45]==transcripts[46])

print(transcripts[45])

100
50
ENST00000371007
yeth
True
True
True
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO